In [1]:
import requests

In [26]:
url = 'https://tle.ivanstanojevic.me/api/tle/25544'
response = requests.get(url)
tle_data = response.json()

In [27]:
line1 = tle_data['line1']
line2 = tle_data['line2']

In [12]:
from skyfield.api import load, wgs84, EarthSatellite
from datetime import datetime, timedelta
import numpy as np

In [28]:

ts = load.timescale()

satellite = EarthSatellite(line1, line2, 'ISS (ZARYA)', load.timescale())

# Calculate positions for the next 24 hours
times = [ts.now() + timedelta(hours=i) for i in range(24)]
latitudes = []
longitudes = []


In [29]:
for t in times:
    geocentric = satellite.at(t)
    subpoint = wgs84.subpoint(geocentric)
    latitudes.append(subpoint.latitude.degrees)
    longitudes.append(subpoint.longitude.degrees)
    at

In [30]:
print(latitudes)

[-10.089841572842165, 45.64102439739891, -44.03756406679326, 7.407230587076771, 32.56409743005419, -51.4467395250772, 24.42689071615701, 16.29105592769182, -48.88515445174619, 39.480997866920305, -1.1322278027681332, -37.831279028788416, 49.7454508975113, -18.476113618829327, -22.445759442538066, 51.02357176319628, -34.46486341591279, -5.3191707297557365, 42.56407943969921, -46.88470570312608, 12.153335680485872, 28.324649934518096, -51.79458418490565, 28.81200759386007]


In [25]:
print(longitudes)

[-99.41043994069202, 105.33565732146575, -51.74440604782134, -173.58804140536876, 45.15666206934608, -115.90496532895187, 105.53010819083305, -18.289438679766057, -176.31555371250647, 29.95671490082955, -89.00302282801358, 123.94888599404314, -36.60098166308789, -168.5389723955683, 62.1286305706891, -97.98744140016493, 112.85689410847279, -5.40243525509273, -157.6801142961992, 43.418388123372, -82.15910623613371, 141.5846585039823, -19.339858921531768, -162.7673837191992]


In [62]:

times = [ts.now() + timedelta(minutes=i) for i in range(70)]
latitudes = []
longitudes = []
altitudes = []

for t in times:
    geocentric = satellite.at(t)
    subpoint = wgs84.subpoint(geocentric)
    latitudes.append(subpoint.latitude.degrees)
    longitudes.append(subpoint.longitude.degrees)
    altitudes.append(subpoint.elevation.m)

In [67]:
mean_motion = satellite.model.no_kozai

In [68]:
orbital_period_minutes = 1440 / mean_motion

In [69]:
times = ts.utc(start_time.utc_datetime(), 
               end_time.utc_datetime() + timedelta(minutes=orbital_period_minutes), 
               timedelta(minutes=5))

In [72]:
from datetime import datetime, timedelta
ts = load.timescale()

start_time = datetime.utcnow()

In [73]:
orbital_period_hours = 1.5  # For example, 90 minutes for the ISS

# Calculate end_time based on the orbital period
end_time = start_time + timedelta(hours=orbital_period_hours)

# Convert start_time and end_time to Skyfield Time objects
start_time_ts = ts.utc(start_time.year, start_time.month, start_time.day, start_time.hour, start_time.minute)
end_time_ts = ts.utc(end_time.year, end_time.month, end_time.day, end_time.hour, end_time.minute)

# Generate a range of times every 5 minutes over the orbital period
times = ts.utc(start_time.year, start_time.month, start_time.day, start_time.hour, range(start_time.minute, end_time.minute + 5, 5))

# Now 'times' should be an iterable sequence of Time objects

In [74]:
latitudes = []
longitudes = []
altitudes = []

for t in times:
    geocentric = satellite.at(t)
    subpoint = wgs84.subpoint(geocentric)
    latitudes.append(subpoint.latitude.degrees)
    longitudes.append(subpoint.longitude.degrees)
    altitudes.append(subpoint.elevation.m)

In [75]:
import folium

# Your Mapbox access token
mapbox_access_token = 'pk.eyJ1IjoiamF5dGlsYWsiLCJhIjoiY2xzbmpmbGgxMDR6MjJsbHJwcWNiZ3MzeiJ9.1NGfoYr6EeGENibTd2ZQKQ'
mapbox_tile_url = f"https://api.mapbox.com/styles/v1/mapbox/streets-v11/tiles/256/{{z}}/{{x}}/{{y}}?access_token={mapbox_access_token}"

# Create a Folium map using Mapbox tiles
m = folium.Map(location=[0, 0], zoom_start=2, tiles=mapbox_tile_url, attr='Mapbox')



In [78]:
folium.PolyLine(
    list(zip(latitudes, longitudes)),
    color="red",
    weight=2.5,
    opacity=1
).add_to(m)

m

In [77]:
import folium

# Use the first position as the current position
current_position = [latitudes[0], longitudes[0]]

# Create a map centered around the current position of the satellite
m = folium.Map(location=current_position, zoom_start=3)

# Add a marker for the current position
folium.Marker(current_position, tooltip='ISS (ZARYA) Current Position').add_to(m)


# Display the map
m

In [19]:
def segment_path(latitudes, longitudes):
    segments = []
    current_segment = [[latitudes[0], longitudes[0]]]

    for lat, lon in zip(latitudes[1:], longitudes[1:]):
        if abs(lon - current_segment[-1][1]) > 180:  # Crossing the antimeridian
            # Finish the current segment and start a new one
            segments.append(current_segment)
            current_segment = [[lat, lon]]
        else:
            current_segment.append([lat, lon])
    
    # Add the last segment
    segments.append(current_segment)
    return segments

segments = segment_path(latitudes, longitudes)


In [44]:
# Create a GeoJSON LineString for the satellite's path
geojson_path = {
    "type": "FeatureCollection",
    "features": [{
        "type": "Feature",
        "geometry": {
            "type": "LineString",
            "coordinates": [[lon, lat, alt] for lat, lon, alt in zip(latitudes, longitudes, altitudes)]
        },
        "properties": {
            "name": "ISS Orbit Path"
        }
    }]
}



In [49]:
# Save the GeoJSON to a file
geojson_filename = '/home/jay/Documents/ENVS456-Web_Mapping_and_Geovisualisation/envs456/labs/st.geojson'
with open(geojson_filename, 'w') as f:
    json.dump(geojson_path, f)

geojson_filename

'/home/jay/Documents/ENVS456-Web_Mapping_and_Geovisualisation/envs456/labs/st.geojson'

In [81]:
from IPython.display import HTML



html_code = """
<!DOCTYPE html>
<html lang='en'>
<head>
    <meta charset='utf-8' />
    <title>Points on a map</title>
    <meta name='viewport' content='width=device-width, initial-scale=1' />
    <script src='https://api.tiles.mapbox.com/mapbox-gl-js/v3.1.0/mapbox-gl.js'></script>
    <link href='https://api.tiles.mapbox.com/mapbox-gl-js/v3.1.0/mapbox-gl.css' rel='stylesheet' />
    <style>
        body { 
            margin: 0; 
            padding: 0; 
        }
        #map { 
            position: absolute; 
            top: 0; 
            bottom: 0; 
            width: 100%; 
        }
    </style>
</head>
<body>
<div id='map'></div>
<script>
    mapboxgl.accessToken = 'pk.eyJ1IjoiamF5dGlsYWsiLCJhIjoiY2xzbmpmbGgxMDR6MjJsbHJwcWNiZ3MzeiJ9.1NGfoYr6EeGENibTd2ZQKQ';
    const map = new mapboxgl.Map({
        container: 'map',
        style: 'mapbox://styles/jaytilak/cltabg4q600ub01pj34qx93lw',
        center: [87.1653773271837, -2.8004583596635726],
        zoom: 1
    });

    map.on('load', function() {
      // Add a marker for the satellite
       var el = document.createElement('div');
       el.className = 'marker'; 
       el.style.backgroundImage = "url('https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQudAU0A027SOaBBB1jw13HG90Z9-LzD92XKoTvfwKeYZvmMjKk7NgbvbdVY-N0f7e3_co&usqp=CAU')"; // Replace this with your actual image URL
       el.style.width = '20px';
       el.style.height = '20px';
       el.style.backgroundSize = 'cover';




       
       var marker = new mapboxgl.Marker(el)
         .setLngLat([87.1653773271837, -2.8004583596635726]) // Set to the satellite's position
         .addTo(map);

       // Add a popup to the marker
       var popup = new mapboxgl.Popup({ offset: 25 })
           .setText('International Space Station Location') // Popup text
           .addTo(map);

      // Associate the popup with the marker
       marker.setPopup(popup);
});
</script>
</body>
</html>
"""
file_name = 'mapbox_map.html'
with open(file_name, 'w') as file:
    file.write(html_code)

## to ensure below map works, please connect to internet

In [4]:
from IPython.display import IFrame


# Display the HTML file in an IFrame
IFrame(src="https://tilak-pentapati.github.io/mapbox_map.html", width='100%', height='500px')

In [5]:
from IPython.display import HTML

HTML("""
<!DOCTYPE html>
<html lang='en'>
<head>
    <meta charset='utf-8' />
    <title>Points on a map</title>
    <meta name='viewport' content='width=device-width, initial-scale=1' />
    <script src='https://api.tiles.mapbox.com/mapbox-gl-js/v3.1.0/mapbox-gl.js'></script>
    <link href='https://api.tiles.mapbox.com/mapbox-gl-js/v3.1.0/mapbox-gl.css' rel='stylesheet' />
    <style>
        body { 
            margin: 0; 
            padding: 0; 
        }
        #map { 
            position: absolute; 
            top: 0; 
            bottom: 0; 
            width: 100%; 
        }
    </style>
</head>
<body>
<div id='map'></div>
<script>
    mapboxgl.accessToken = 'pk.eyJ1IjoiamF5dGlsYWsiLCJhIjoiY2xzbmpmbGgxMDR6MjJsbHJwcWNiZ3MzeiJ9.1NGfoYr6EeGENibTd2ZQKQ';
    const map = new mapboxgl.Map({
        container: 'map',
        style: 'mapbox://styles/jaytilak/cltabg4q600ub01pj34qx93lw',
        center: [87.1653773271837, -2.8004583596635726],
        zoom: 1
    });

    map.on('load', function() {
      // Add a marker for the satellite
       var el = document.createElement('div');
       el.className = 'marker'; 
       el.style.backgroundImage = "url('https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQudAU0A027SOaBBB1jw13HG90Z9-LzD92XKoTvfwKeYZvmMjKk7NgbvbdVY-N0f7e3_co&usqp=CAU')"; // Replace this with your actual image URL
       el.style.width = '20px';
       el.style.height = '20px';
       el.style.backgroundSize = 'cover';




       
       var marker = new mapboxgl.Marker(el)
         .setLngLat([87.1653773271837, -2.8004583596635726]) // Set to the satellite's position
         .addTo(map);

       // Add a popup to the marker
       var popup = new mapboxgl.Popup({ offset: 25 })
           .setText('International Space Station Location') // Popup text
           .addTo(map);

      // Associate the popup with the marker
       marker.setPopup(popup);
});
</script>
</body>
</html>

""")

In [7]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd

# Example DataFrames
df1 = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})
df2 = pd.DataFrame({'A': [7, 8, 9], 'B': [10, 11, 12]})
# Add more DataFrames as needed...

# Assuming you have a list or dictionary of DataFrames
dfs = {'Satellite 1': df1, 'Satellite 2': df2}  # Extend this with all 24 DataFrames

# Create a Dropdown widget
dropdown = widgets.Dropdown(
    options=list(dfs.keys()),
    value=list(dfs.keys())[0],  # Default value
    description='Satellite:',
    disabled=False,
)

# Create an Output widget to display DataFrame
output = widgets.Output()

# Dropdown event handler
def dropdown_eventhandler(change):
    output.clear_output()  # Clear the previous DataFrame
    with output:
        display(dfs[change.new])  # Display the selected DataFrame

# Observe the dropdown
dropdown.observe(dropdown_eventhandler, names='value')

# Display the widgets
display(dropdown, output)

# Initially display the first DataFrame
with output:
    display(dfs[dropdown.value])

Dropdown(description='Satellite:', options=('Satellite 1', 'Satellite 2'), value='Satellite 1')

Output()